# RTS Simulation Engine
#### Development Notes:
Please add **"TBI"** as comment to the sections under development (e.g. _# To be randomized (TBI)_)

In [1]:
# Libraries
import math
import time
import random
import os
import platform
import pathlib
import subprocess
import numpy as np
from matplotlib import pyplot as plt
from openpyxl import Workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from datetime import datetime
from tkinter import *
from tkinter import messagebox  
from PIL import Image, ImageTk

In [2]:
# Global variables
logFileName = "sim_log.txt"
logFilePath = '../Blender/'
roadXLength = 0
LABEL_FONT = 'Times New Roman'
LABEL_SIZE = 12
SCALE_LENGTH = 180
PADX = 10
PADY = 5
IPADY = 5
PADX_SLIDER = 20
PADY_SLIDER = 4
RIGHT_BG = '#95A5A6'
data = None

In [3]:
# Curve functions
def arclength(f, a, b, tol=1e-6):
    nSteps = 1                                       # number of steps to compute
    oldLength = 1.0e20
    length = 1.0e10
    while abs(oldLength - length) >= tol:
        nSteps *= 2
        y = f(a)
        xDelta = (b - a) / nSteps                    # space between x-values
        oldLength = length
        length = 0
        for i in range(1, nSteps + 1):
            yPrev = y                                # previous function value
            y = f(a + i * (b - a) / nSteps)          # new function value
            length += math.hypot(xDelta, y - yPrev)  # length of small line segment
    return length


def findNextX(f, currentX, targetLength, step, errorTol):
    nextXold =  nextX = currentX
    calculatedLength = 0

    while(targetLength >= calculatedLength):
        nextX += step*2
        calculatedLength = arclength(f, currentX, nextX, errorTol)

    while abs(targetLength-calculatedLength) > errorTol:
        middle = (nextX+nextXold)/2
        calculatedLength = arclength(f, currentX, middle, errorTol)
        if(calculatedLength < targetLength):
            nextXold = middle
        else:
            nextX = middle

    return nextX

In [4]:
# Degree Calc.
def calculate_angle(item_x, lane, measure):
    derivative_eq = np.polyder(lane.equation, 1)
    slope = derivative_eq(item_x)
    radian_angle = math.atan(slope)
    if measure == 'radian':
        return radian_angle
    degree_angle = math.degrees(radian_angle)
    return degree_angle

In [5]:
class Logger:
    
    def __init__(self, road, interval, day_time, potholes, lights, x_start, x_end, y_start, y_end):
        self.day_time = day_time
        self.vehicles = list()
        self.potholes = potholes
        self.lights = lights
        self.road = road
        self.interval = interval
        self.x_start = x_start
        self.y_start = y_start
        self.x_end = x_end
        self.y_end = y_end
    
    def addvehicle(self, vehicle):
        self.vehicles.append(vehicle)
        
        
    def addPotholes(self, pothole):
        self.potholes.append(pothole)
    
    
    def outputToFile(self):
        file_name_with_path = logFilePath + logFileName 
        f = open(file_name_with_path, "w")
        
        # Log road
        roadLocationList = list()
        
        roadEquation = self.road.lanes[1].equation
        x_values = np.arange(0,roadXLength,self.interval).tolist()+[roadXLength]
        
        for x in x_values:
            roadLocationList.append([x,round(roadEquation(x),2),0])
        
        tripleStrList = list()
        for triple in roadLocationList:
            tripleStr = str(triple[0]) + "," + str(triple[1]) + ",0," + str(triple[2])
            tripleStrList.append(tripleStr)
        roadStr = self.road.name + ' '  + ' '.join(tripleStrList)
        f.write(roadStr + "\n")
        
        # Log lanes
        laneCount = len(self.road.lanes)
        f.write(f'Lane {laneCount}\n')
        
        #Log wall
        f.write(f'Wall {calculate_angle(self.x_start, self.road.lanes[0], "radian")} {calculate_angle(self.x_end, self.road.lanes[0], "radian")}\n')
        
        # Log lights
        #lightStr = ' '.join(tripleStrList)
        #f.write(lightStr + "\n")
        
        potholes_logs = ''
        # Log potholes with locations.
        for pothole in self.potholes:
            potholes_logs += f' {pothole.size.upper()},{pothole.x},{pothole.y},0'
            
        f.write('Defect' + potholes_logs + '\n')
        
        # Log Traffic Lights
        lights_log = ''
        for light in self.lights:
            lights_log += f'{light.frequency},{light.x},{light.y},0,{calculate_angle(light.x, light.lane, "radian") }'
        f.write(f'TrafficLight {lights_log}\n')
        
        # Log daytime.
        f.write('Time ' + self.day_time[0] + '\n')
        
        # Log vehicles with locations and radian angles
        for vehicle in self.vehicles:
            locationStrList = list()
            for loc in vehicle.location_log:
                locationStrList.append(str(loc[0]) + "," + str(loc[1]) + ",0," + str(loc[2]))
            
                
            line = f'{str(vehicle)} {str(vehicle.driver)} {vehicle.driver.age} {vehicle.cycle_offset} '      
            line += " ".join(locationStrList) + "\n"
            f.write(line)
        
        
        # Close file.
        f.close()

In [6]:
class TrafficLight:
    status = 'red'
    
    def __init__(self, x, y, z, frequency):
        self.x = x
        self.y = y
        self.z = z
        self.frequency = frequency
        self.lane = None
    
    
    def set_lane(self, lane):
        self.lane = lane
        
        
    def change(self):
        if self.status == 'red':
            self.status = 'green'
        else:
            self.status = 'red'

In [7]:
class Lane:    
    
    def __init__(self, equation, x_begin, y_begin):
        self.parent_road = None
        self.equation = equation
        self.x_begin = x_begin
        self.y_begin = y_begin
        self.vehicles = []
        self.potholes = []
        self.lights = []
        # Adjacent lanes
        self.left_lane = None
        self.right_lane = None
        
        
    def set_left_lane(self, lane):
        self.left_lane = lane
        
        
    def set_right_lane(self, lane):
        self.right_lane = lane
    
    
    def set_pothole(self, pothole):
        self.potholes.append(pothole)
        
        
    def set_road(self, road):
        self.parent_road = road
    
    
    def set_light(self, light):
        self.lights.append(light)
        
        
    def addToLane(self, vehicle):
        self.vehicles.append(vehicle)
        vehicle.set_lane(self)
        
        
    def remove_vehicle(self, vehicle):
        self.vehicles.remove(vehicle)
        
        
    def isEmpty(self):
        return True if not self.vehicles else False
                
        
    def __str__(self):
        strprt = ""
        for vehicle in self.vehicles:
            strprt += vehicle + ":" + str(vehicle.location) + "  "
        return strprt

In [8]:
class Road:    
    
    def __init__(self, name, lanes, lane_distance):
        self.name = name
        self.lanes = lanes
        self.lane_distance = lane_distance # Assumed that every lane of road have the same width.
        self.collided_vehicles = []
    
    
    def isEmpty(self):
        for lane in self.lanes:
            if not lane.isEmpty():
                return False
        return True

In [9]:
class Vehicle:
    
    def __init__(self, vehicle_id, driver, initialSpeed): # initloc last radian
        self.vehicle_id = vehicle_id
        self.driver = driver
        self.speed = initialSpeed
        self.location = None
        self.location_log = []
        self.combo = 0.0 # This variable is used to simulate smooth braking
        self.lane = None
        self.is_collided = False
        self.mode = 0    # 1 is for lane-changing, 0 is for driving forward
        self.switch_lane = None
        self.switch_iter = 0
        self.speed_log = []
    
    
    def set_lane(self, lane):
        self.lane = lane
    
    
    def set_cycle_offset(self, cycle_offset):
        self.cycle_offset = cycle_offset
    
    
    def set_location(self, location):
        self.location = location
        self.logPosition()
        
        
    def accelerate(self):
        self.combo = 0  # Reset the brake combo
        if self.speed >= self.max_speed:
            self.speed = self.max_speed
        else:
            self.speed += self.max_acc * driver.acc_coef
        self.__move()
        
        
    def decelerate(self):       
        speed_reduction = driver.dec_coef * self.max_brake + self.combo
        self.combo += driver.combo_increment
        if speed_reduction > self.max_brake:
            speed_reduction = self.max_brake
        self.speed -= speed_reduction
        
        if self.speed < self.min_speed:
            self.speed = self.min_speed
            self.combo = 0
        self.__move()
    
    
    def __move(self):
        current_x = self.location[0]
        self.location[0] = findNextX(self.lane.equation, current_x, self.speed, 1, 1e-2)
        self.location[1] = self.lane.equation(self.location[0])
        self.location[2] = self.calculate_angle('radian')
        self.logPosition()
        self.speed_log.append(self.speed)
    
    
    def __move_lane(self, shift):
        current_x = self.location[0]
        self.location[0] = findNextX(self.lane.equation, current_x, self.speed, 1, 1e-2)
        self.location[1] = self.lane.equation(self.location[0]) + shift
        self.location[2] = self.calculate_angle('radian')
        self.logPosition()
        self.speed_log.append(self.speed)
        
        
    def logPosition(self):
        self.location_log.append(list(self.location))
        
    
    def search_front_vehicle(self):        
        front_vehicles = [vehicle for vehicle in self.lane.vehicles if vehicle.location[0] > self.location[0]]
        for vehicle in front_vehicles:
            self_x = self.location[0]
            self_y = self.location[1]
            vehicle_x = vehicle.location[0]
            vehicle_y = vehicle.location[1]
            euclidian_dist = math.dist([self_x, self_y], [vehicle_x, vehicle_y])
            if euclidian_dist <= self.driver.vision: #and self.calc_relative_speed(vehicle) > 0: # This should not be zero, should be tuned regarding to driver type etc.
                return True
        return False
    
    
    def search_front_light(self):
        front_lights = [light for light in self.lane.lights if light.location[0] > self.location[0]]
        for light in front_lights:
            self_x = self.location[0]
            self_y = self.location[1]
            light_x = light.location[0]
            light_y = light.location[1]
            euclidian_dist = math.dist([self_x, self_y], [light_x, light_y])
            if euclidian_dist <= self.driver.vision: #and self.calc_relative_speed(vehicle) > 0: # This should not be zero, should be tuned regarding to driver type etc.
                return True
        return False
        
        
    def calc_relative_speed(self, vehicle):
        return self.speed - vehicle.speed
    
    
    def check_mirrors(self):
        lanes = [self.lane.left_lane, self.lane.right_lane]
        for lane in lanes:
            if lane is not None:
                #back_vehicles = [vehicle for vehicle in lane.vehicles if vehicle.location[0] < self.location[0]]
                check = True
                for vehicle in lane.vehicles:
                    self_x = self.location[0]
                    self_y = self.location[1]
                    vehicle_x = vehicle.location[0]
                    vehicle_y = vehicle.location[1]
                    euclidian_dist = math.dist([self_x, self_y], [vehicle_x, vehicle_y])
                    rel_speed = self.calc_relative_speed(vehicle)
                    x_diff = abs(self_x - vehicle_x)
                    if euclidian_dist <= self.driver.vision:
                        # back control
                        if vehicle.location[0] < self.location[0]:                       
                            if rel_speed < 0 and abs(rel_speed)*5 >= x_diff:
                                check = False
                        else:
                            # front control
                            if rel_speed > 0 and abs(rel_speed)*5 <= x_diff:
                                check = False 
                        #if self.calc_relative_speed(vehicle) < 0:# This should not be zero it should be tuned regarding to driver type etc.
                         #   check = False
                if check:
                    return lane
        return None
        
        
    def decide(self):
        self.check_collision()
        if not self.is_collided:
            # Lane changing mode.
            if self.mode:
                shift = self.switch_lane.parent_road.lane_distance
                if self.switch_lane == self.lane.right_lane:
                    shift *= -1
                self.switch_iter += 1
                self.__move_lane(shift / 5 * self.switch_iter)
                if self.switch_iter == 5:                
                    if self.switch_lane == self.lane.left_lane:
                        self.lane.remove_vehicle(self)
                        self.lane.left_lane.addToLane(self)
                    else:                    
                        self.lane.remove_vehicle(self)
                        self.lane.right_lane.addToLane(self)
                    self.mode = 0
                    self.switch_iter = 0
                    self.switch_lane = None
            else:
                # Check any vehicle nearby in front of self vehicle.
                if self.search_front_vehicle():
                    lane_to_switch = self.check_mirrors()

                    # Check whether any lane is available. If there is no available 
                    # lane then decelerate.
                    if lane_to_switch is not None:
                        self.mode = 1
                        self.switch_lane = lane_to_switch               
                    else:
                        self.decelerate()

                self.accelerate()
                    
        
    def check_collision(self):
               
        left_boundary = self.location[1] - self.width / 2
        right_boundary = self.location[1] + self.width / 2
        up_boundary = self.location[0] + self.length / 2
        down_boundary = self.location[0] - self.length / 2
        
        current_road = self.lane.parent_road
        for lane in current_road.lanes:
            for vehicle in lane.vehicles:
                if vehicle.vehicle_id != self.vehicle_id and (down_boundary <= vehicle.location[0] <= up_boundary and left_boundary <= vehicle.location[1] <= right_boundary):
                    self.is_collided = True
                    vehicle.is_collided = True
                    self.speed = 0
                    vehicle.speed = 0
                    current_road.collided_vehicles.append(self)
                    current_road.collided_vehicles.append(vehicle)
                    return
        self.is_collided = False
        
        
    def calculate_angle(self, measure):
        derivative_eq = np.polyder(self.lane.equation, 1)
        slope = derivative_eq(self.location[0])
        radian_angle = math.atan(slope)
        if measure == 'radian':
            return radian_angle
        degree_angle = math.degrees(radian_angle)
        return degree_angle

Sedan: 2.35 x 1 <br/>
Minibüs: 2.6 x 1.15 <br/>
Kamyon: 3.4 x 1.4 <br/>
Otobüs: 6.3 x 1.4 <br/>
Blender 1 metresi = 1.792 meter <br/>

In [10]:
class Car(Vehicle):
    max_speed = 20.0
    min_speed = 0.0
    max_acc = max_speed / 10
    max_brake = 5
    width = 1
    length = 2.35
    
    
    def __init__(self, vehicle_id, driver, initialSpeed):
        super().__init__(vehicle_id, driver, initialSpeed)
        
        
    def __str__(self):
        return 'Sedan'

In [11]:
class Truck(Vehicle):
    max_speed = 14.0
    min_speed = 0.0
    max_acc = max_speed / 20
    max_brake = 1.5
    width = 1.4
    length = 3.4
    
    
    def __init__(self, vehicle_id, driver, initialSpeed):
        super().__init__(vehicle_id, driver, initialSpeed)
        
        
    def __str__(self):
        return 'Truck'

In [12]:
class Bus(Vehicle):
    max_speed = 20.0
    min_speed = 0.0
    max_acc = max_speed / 15
    max_brake = 3
    width = 1.4
    length = 6
    
    
    def __init__(self, vehicle_id, driver, initialSpeed):
        super().__init__(vehicle_id, driver, initialSpeed)
    
    
    def __str__(self):
        return 'Bus'

In [13]:
class Van(Vehicle):
    max_speed = 20.0
    min_speed = 0.0
    max_acc = max_speed / 10
    max_brake = 5
    width = 1.15
    length = 2.6
    
    
    def __init__(self, vehicle_id, driver, initialSpeed):
        super().__init__(vehicle_id, driver, initialSpeed)
        
        
    def __str__(self):
        return 'Van'

In [14]:
class Driver():
    
    def __init__(self, age):
        self.age = age
        self.vision = self.__set_vision()
        
        
    def __set_vision(self):
        avg_vision = 90
        return (avg_vision * 0.7 + avg_vision * 0.3 * (1 / (self.age - 18))) * self.vision_coef

In [15]:
class RookieDriver(Driver):
    dec_coef = 0.7
    combo_increment = 0.8
    acc_coef = 0.3
    vision_coef = 0.5
    
    
    def __init__(self, age):
        super().__init__(age)
        
        
    def __str__(self):
        return 'Rookie'

In [16]:
class HastyDriver(Driver):
    dec_coef = 0.9
    combo_increment = 0.7
    acc_coef = 0.9
    vision_coef = 0.6
    
    def __init__(self, age):
        super().__init__(age)
        
    def __str__(self):
        return 'Hasty'

In [17]:
class ProfessionalDriver(Driver):
    dec_coef = 0.8
    combo_increment = 0.1
    acc_coef = 0.3
    vision_coef = 1
    
    def __init__(self, age):
        super().__init__(age)
        
        
    def __str__(self):
        return 'Professional'

In [18]:
class Pothole:
    
    def __init__(self, pothole_id, x, y, size):
        self.pothole_id = pothole_id
        self.x = x
        self.y = y
        self.size = size
        self.__initialize_slowness_coef(size)
    
    
    def set_lane(self, lane):
        self.lane = lane
        
        
    def __initialize_slowness_coef(self, size):
        size = size.lower()
        self.slow_coef = 0.0
        if size == 's':
            slow_coef = 0.1
        elif size == 'm':
            slow_coef = 0.2
        elif size == 'l':
            slow_coef = 0.4

In [19]:
class ExcelReporter:
    
    @staticmethod
    def report(vehicles):
        wb = Workbook()
        ws = wb.active
        ws.title = 'Report'

        all_params = []
        
        for key in data:
            all_params.append([key, str(data[key])])

        vehicle_params = []
        
        for vehicle in vehicles:
            vehicle_params.append([str(vehicle.driver), str(vehicle), str(vehicle.driver.age)])


        table1 = Table(displayName="RtsTableGeneral", ref=f"A1:{chr(65 + len(all_params) - 1)}{len(all_params[0])}")
        table_style = TableStyleInfo(name="TableStyleMedium1", showFirstColumn=True, showLastColumn=True)
        table1.tableStyleInfo = table_style
        ws.add_table(table1)

        second_table_index = 7

        table2 = Table(displayName="RtsTableDrivers", ref=f"A{second_table_index}:{chr(65 + len(vehicle_params[0]) - 1)}{len(vehicle_params) + second_table_index}")
        table_style2 = TableStyleInfo(name="TableStyleMedium2", showFirstColumn=True, showLastColumn=True)
        table2.tableStyleInfo = table_style2
        ws.add_table(table2)

        for i, col in enumerate(all_params):
            for j, row in enumerate(all_params[i]):
                print(i+1, j+1 , row)
                ws.cell(column=i+1, row=j+1, value=f"{row}")


        ws.cell(column=1, row=7, value="Driver")
        ws.cell(column=2, row=7, value="Vehicle")
        ws.cell(column=3, row=7, value="Age")
        
        for i, row in enumerate(vehicle_params):
            for j, col in enumerate(vehicle_params[i]):
                print(j+1, i+8 , col)
                ws.cell(column=j+1, row=i+8, value=f"{col}")

        date = datetime.now().strftime("%d-%m-%Y_%I-%M-%S_%p")
        print(date)

        filename = f'RTS_Analysis_Report{date}.xlsx'
        wb.save(filename)

In [20]:
def GUI():
    window = Tk()
    w, h = window.winfo_screenwidth(), window.winfo_screenheight()
    window.geometry("%dx%d+0+0" % (w, h))
    window.columnconfigure(0, weight=1)
    window.rowconfigure(0, weight=1)
    window.title("RTS")
    window.iconbitmap('Assets/rts.ico')
    window.configure(bg="grey")
    #window.resizable(False, False)
    
    main_frame = Frame(window)
    main_frame.grid(row=0, column=0, sticky="nsew")
    main_frame.columnconfigure(0, weight=1)
    main_frame.columnconfigure(1, weight=1)
    main_frame.rowconfigure(0, weight=1)

    right_frame = Frame(main_frame, bg=RIGHT_BG)
    right_frame.grid(row=0, column=1, sticky="nswe")

    # for i in range(4):
    #     right_frame.rowconfigure(i, weight=1)
    #     right_frame.columnconfigure(i, weight=1)
    # for i in range(3):
    #     right_frame.rowconfigure(i, weight=1)
    #     right_frame.columnconfigure(i+4, weight=3)
    # right_frame.rowconfigure(7, weight=1)
    # right_frame.columnconfigure(7, weight=1)


    left_frame = Frame(main_frame)
    left_frame.grid(row=0, column=0, sticky="nswe")

    age_label = Label(left_frame, text='Mean Driver Age:', font=(LABEL_FONT, LABEL_SIZE, 'bold'))
    age_label.grid(row=0, column=0, padx=PADX, pady=PADY)
    age_var = StringVar(value='30')
    age_entry = Entry(left_frame, textvariable=age_var)
    age_entry.grid(row=0, column=1, padx=PADX, pady=PADY, ipady=IPADY)
    driver_label = Label(left_frame, text="Number of Drivers:", font=(LABEL_FONT, LABEL_SIZE, 'bold'))
    driver_label.grid(row=1, column=0, padx=PADX, pady=PADY)
    driver_var = StringVar(value='1')
    driver_entry = Entry(left_frame, textvariable=driver_var)
    driver_entry.grid(row=1, column=1, padx=PADX, pady=PADY, ipady=IPADY)

    label_rookie = Label(left_frame, text="Rookie Drivers", font=(LABEL_FONT, LABEL_SIZE))
    label_rookie.grid(row=2, column=0, padx=PADX, pady=PADY)
    slider_rookie = Scale(left_frame, from_=0, to=0, orient=HORIZONTAL, length=SCALE_LENGTH)
    slider_rookie.grid(row=2, column=1, padx=PADX_SLIDER, pady=PADY_SLIDER)
    label_hasty = Label(left_frame, text="Hasty Drivers", font=(LABEL_FONT, LABEL_SIZE))
    label_hasty.grid(row=3, column=0, padx=PADX, pady=PADY)
    slider_hasty = Scale(left_frame, from_=0, to=0, orient=HORIZONTAL, length=SCALE_LENGTH)
    slider_hasty.grid(row=3, column=1, padx=PADX_SLIDER, pady=PADY_SLIDER)
    label_pro = Label(left_frame, text="Professional Drivers", font=(LABEL_FONT, LABEL_SIZE))
    label_pro.grid(row=4, column=0, padx=PADX, pady=PADY)
    slider_pro = Scale(left_frame, from_=0, to=0, orient=HORIZONTAL, length=SCALE_LENGTH)
    slider_pro.grid(row=4, column=1, padx=PADX_SLIDER, pady=PADY_SLIDER)

    label_sedan = Label(left_frame, text="Sedan Count", font=(LABEL_FONT, LABEL_SIZE))
    label_sedan.grid(row=5, column=0, padx=PADX_SLIDER, pady=PADY)
    slider_sedan = Scale(left_frame, from_=0, to=0, orient=HORIZONTAL, length=SCALE_LENGTH)
    slider_sedan.grid(row=5, column=1, padx=PADX_SLIDER, pady=PADY_SLIDER)
    label_van = Label(left_frame, text="Van Count", font=(LABEL_FONT, LABEL_SIZE))
    label_van.grid(row=6, column=0, padx=PADX, pady=PADY)
    slider_van = Scale(left_frame, from_=0, to=0, orient=HORIZONTAL, length=SCALE_LENGTH)
    slider_van.grid(row=6, column=1, padx=PADX_SLIDER, pady=PADY_SLIDER)
    label_bus = Label(left_frame, text="Bus Count", font=(LABEL_FONT, LABEL_SIZE))
    label_bus.grid(row=7, column=0, padx=PADX, pady=PADY)
    slider_bus = Scale(left_frame, from_=0, to=0, orient=HORIZONTAL, length=SCALE_LENGTH)
    slider_bus.grid(row=7, column=1, padx=PADX_SLIDER, pady=PADY_SLIDER)
    label_truck = Label(left_frame, text="Truck Count", font=(LABEL_FONT, LABEL_SIZE))
    label_truck.grid(row=8, column=0, padx=PADX, pady=PADY)
    slider_truck = Scale(left_frame, from_=0, to=0, orient=HORIZONTAL, length=SCALE_LENGTH)
    slider_truck.grid(row=8, column=1, padx=PADX_SLIDER, pady=PADY_SLIDER)
    
    driver_button = Button(left_frame, text='submit', command=lambda:check_entry(driver_var.get(), 'driver', slider_rookie, slider_pro, slider_hasty, slider_sedan, slider_bus, slider_van, slider_truck))
    driver_button.grid(row=1, column=3, padx=PADX, pady=PADY)
    
    defect_label = Label(left_frame, text='Number of Defects:', font=(LABEL_FONT, LABEL_SIZE, 'bold'))
    defect_label.grid(row=9, column=0, padx=PADX, pady=PADY)
    defect_var = StringVar(value='0')
    defect_entry = Entry(left_frame, textvariable=defect_var)
    defect_entry.grid(row=9, column=1, padx=PADX, pady=PADY, ipady=IPADY)

    small_defect_label = Label(left_frame, text="Small Defects", font=(LABEL_FONT, LABEL_SIZE))
    small_defect_label.grid(row=10, column=0, padx=PADX, pady=PADY)
    small_defect_slider = Scale(left_frame, from_=0, to=0, orient=HORIZONTAL, length=SCALE_LENGTH)
    small_defect_slider.grid(row=10, column=1, padx=PADX_SLIDER, pady=PADY_SLIDER)
    medium_defect_label = Label(left_frame, text="Medium Defects", font=(LABEL_FONT, LABEL_SIZE))
    medium_defect_label.grid(row=11, column=0)
    medium_defect_slider = Scale(left_frame, from_=0, to=0, orient=HORIZONTAL, length=SCALE_LENGTH)
    medium_defect_slider.grid(row=11, column=1, padx=PADX_SLIDER, pady=PADY_SLIDER)
    large_defect_label = Label(left_frame, text="Large Defects", font=(LABEL_FONT, LABEL_SIZE))
    large_defect_label.grid(row=12, column=0)
    large_defect_slider = Scale(left_frame, from_=0, to=0, orient=HORIZONTAL, length=SCALE_LENGTH)
    large_defect_slider.grid(row=12, column=1, padx=PADX_SLIDER, pady=PADY_SLIDER)
    
    defect_button = Button(left_frame, text='submit', command=lambda:check_entry(defect_var.get(), 'defect', small_defect_slider, medium_defect_slider, large_defect_slider,))
    defect_button.grid(row=9, column=3, padx=PADX, pady=PADY)

    light_label = Label(left_frame, text='Traffic Lights:', font=(LABEL_FONT, LABEL_SIZE, 'bold'))
    light_label.grid(row=13, column=0, padx=PADX, pady=PADY)
    light_var = StringVar(value='0')
    light_entry = Entry(left_frame, textvariable=light_var)
    light_entry.grid(row=13, column=1, padx=PADX, pady=PADY, ipady=IPADY)

    #.resize((700, 300))
    rts_image = Image.open("Assets/RTS.png")
    rts_img = ImageTk.PhotoImage(rts_image.resize((700, 300)))
    rts_photo = Label(right_frame, image=rts_img)
    rts_photo.image = rts_img
    rts_photo.grid(row=0, column=0, padx=PADX, pady=PADY)
    
    road_list = ['Kopuz - Ozyegin 1', 'Kopuz - Ozyegin 2', 'Kopuz - Ozyegin 3',]
    road_var = StringVar(value='Kopuz - Ozyegin 1')
    
    image = Image.open("Assets/Section1.png")
    img = ImageTk.PhotoImage(image.resize((700, 300)))
    photo = Label(right_frame, image=img)
    photo.image = img
    photo.grid(row=2, column=0, padx=PADX, pady=PADY)
    
    road_dropdown = OptionMenu(right_frame, road_var, *road_list, command=lambda _:refresh(road_var.get(), photo, right_frame))
    road_dropdown.grid(row=1, column=0, padx=PADX, pady=PADY)

    lane_label = Label(right_frame, text='Number of Lanes:', font=(LABEL_FONT, LABEL_SIZE, 'bold'), bg=RIGHT_BG)
    lane_label.grid(row=3, column=0, padx=PADX, pady=PADY)
    lane_var = IntVar(value=2)

    radio2 = Radiobutton(right_frame, text='2', variable=lane_var, value=2, bg=RIGHT_BG)
    radio2.grid(row=3, column=1, padx=PADX-3, pady=PADY, ipady=IPADY)
    radio3 = Radiobutton(right_frame, text='3', variable=lane_var, value=3, bg=RIGHT_BG)
    radio3.grid(row=3, column=2, padx=PADX-3, pady=PADY, ipady=IPADY)

    time_label = Label(right_frame, text='Time of Day:', font=(LABEL_FONT, LABEL_SIZE, 'bold'), bg=RIGHT_BG)
    time_label.grid(row=4, column=0, padx=PADX, pady=PADY)
    time_var = StringVar(value='Day')

    radio_day = Radiobutton(right_frame, text='Day', variable=time_var, value='D', bg=RIGHT_BG)
    radio_day.grid(row=4, column=1, padx=PADX, pady=PADY, ipady=IPADY)
    radio_night = Radiobutton(right_frame, text='Night', variable=time_var, value='N', bg=RIGHT_BG)
    radio_night.grid(row=4, column=2, padx=PADX, pady=PADY, ipady=IPADY)

    finish_button = Button(right_frame, text='Run Simulation', bg='light green', command=lambda:finish(age_var, driver_var, slider_rookie, slider_pro, slider_hasty, slider_sedan, slider_van, slider_bus, slider_truck, defect_var, small_defect_slider, medium_defect_slider, large_defect_slider, light_var, road_var, lane_var, time_var))
    finish_button.grid(row=5, column=0, padx=PADX, pady=PADY)

    window.mainloop()
    

#Methods

def finish(age_var, driver_var, slider_rookie, slider_pro, slider_hasty,
            slider_sedan, slider_van, slider_bus, slider_truck,
            defect_var, small_defect_slider, medium_defect_slider, large_defect_slider,
            light_var, road_var, lane_var, time_var):
    
    warning_message = "Sliders' values might be wrong!"
    
    int_age = int(age_var.get())
    if int_age < 18:
        int_age = 18
    elif int_age > 60:
        int_age = 60
        
    int_rookie = int(slider_rookie.get())
    int_pro = int(slider_pro.get())
    int_hasty = int(slider_hasty.get())
    driver_slider_total =  int_rookie + int_hasty + int_pro
    int_driver = int(driver_var.get())
    if driver_slider_total != int_driver :
        messagebox.showerror(title='Input Error!', message=f'{warning_message}(Driver)')
        return
        
    int_sedan = int(slider_sedan.get())
    int_van = int(slider_van.get())
    int_bus = int(slider_bus.get())
    int_truck = int(slider_truck.get())
    vehicle_slider_total = int_bus + int_van + int_truck + int_sedan
    if vehicle_slider_total != int_driver:
        messagebox.showerror(title='Input Error!', message=f'{warning_message}(Vehicle)')
        return
    
    int_s_defect = int(small_defect_slider.get())
    int_m_defect = int(medium_defect_slider.get())
    int_l_defect = int(large_defect_slider.get())
    defect_slider_total = int_s_defect + int_m_defect + int_l_defect
    int_defect = int(defect_var.get())
    if defect_slider_total != int_defect:
        messagebox.showerror(title='Input Error!', message=f'{warning_message}(Defect)')
        return
        
    int_light = int(light_var.get())
    if int_light < 0:
        int_light = 0
        
    str_road = road_var.get()
    int_lane = int(lane_var.get())
    str_time = time_var.get()
    
    global data
    data = {
        'Mean Age': int_age,
        'Driver': int_driver,
        'Rookie': int_rookie,
        'Professional': int_pro,
        'Hasty': int_hasty,
        'Sedan': int_sedan,
        'Van': int_van,
        'Bus': int_bus,
        'Truck': int_truck,
        'Defect': int_defect,
        'Small Defect': int_s_defect,
        'Medium Defect': int_m_defect,
        'Large Defect': int_l_defect,
        'Traffic Light': int_light,
        'Road Part': str_road,
        'Lane': int_lane,
        'Time of Day': str_time
    }
    
    slider_bus.master.master.master.destroy()
    
    
def check_entry(entry, identifier, *sliders):
    message = ''
    int_entry = 1
    print(entry, identifier)
    try:
        int_entry = int(entry)
    except:
        message = 'Input must be a number!'
        
    if identifier == 'driver' and int_entry <= 0:
        message = 'Driver and vehicle count cannot be less then one!'
    elif identifier == 'defect' and int_entry < 0:
        message = 'Defect count cannot be less then zero!'
    elif identifier == 'age' and int_enrty < 18 and int_entry > 60:
        message = 'Mean age must be determined between 18 and 60'
    if message != '':
        messagebox.showerror(title='Input Error!', message=message)
    else:

        if identifier == 'driver':
            update_scalers(entry, sliders[0], sliders[1], sliders[2], sliders[3], sliders[4], sliders[5], sliders[6])
        elif identifier == 'defect':
            update_scalers(entry, sliders[0], sliders[1], sliders[2])
            
        messagebox.showinfo(title='Info', message='Value successfully changed!')
        
    
def update_scalers(new_val, *scalers):
    for scaler in scalers:
        scaler.set(0)
        scaler.configure(to=new_val)


def refresh(road_var, photo, right_frame):
    num = road_var[-1]
    image = Image.open(f"Assets/Section{num}.png")
    img = ImageTk.PhotoImage(image.resize((700, 300)))
    photo = Label(right_frame, image=img)
    photo.image = img
    photo.grid(row=2, column=0, padx=PADX, pady=PADY)
    
    
def clear_frame():
    for widgets in right_frame.winfo_children():
        widgets.destroy()

In [21]:
class Blender:
    
    @staticmethod
    def run():
        os_type = platform.system()
        #blender_exe_path = 'C:\\"Program Files"\\"Blender Foundation"\\"Blender 2.90"\\blender.exe'
        #script_path = pathlib.Path().parent.absolute()  # __file__ does not work in Jupyter Notebook
        #script_path = pathlib.Path(__file__).parent.absolute()
        blender_script_path = os.path.abspath('../Blender/blender_script.py')
        blend_path = os.path.abspath('../Blender/SimulationAnimator.blend')
        command = f'blender {blend_path} --python {blender_script_path}'

        print(os.getcwd())

        if os_type == 'Windows':
            os.chdir(r'C:\Program Files\Blender Foundation\Blender 2.90')
        elif os_type == 'Linux':
            os.chdir(r'/usr/share/blender/2.90')

        print(os.getcwd())
        subprocess.Popen(['blender', blend_path, '--python', blender_script_path])

In [22]:
class SimulationEngine:
    cycle_count = 0
    vehicles = []
    __instance = None
    
    def __init__(self):
        if SimulationEngine.__instance == None:
            SimulationEngine.__instance = self
            
            
    @staticmethod
    def get_instance():
        if SimulationEngine.__instance == None:
            SimulationEngine()
        return SimulationEngine.__instance
    
    
    def get_vehicles(self):
        return self.vehicles
    
    
    def set_day_time(self, day_time):
        self.day_time = day_time
        
        
    def set_road(self, road):
        self.road = road
        
        
    def set_drivers(self, drivers):
        self.drivers = drivers
    
    
    def update_drivers_vision(self):
        # Update driver's vision regarding to daytime.
        for driver in self.drivers:
            if self.day_time == 'D':
                driver.vision = driver.vision
            else:
                driver.vision = driver.vision * 0.75
    
    
    def lights_control(self):
        for lane in self.road.lanes:
            for light in lane.lights:
                if cycle_count % light.frequency == 0:
                    light.change()
                
                
    def move_vehicles(self):
        done_vehicles = 0
        self.update_drivers_vision()
        vehicle_counter = 0
        vehicle_types = ['Sedan', 'Van', 'Bus', 'Truck']
        sedans = data['Sedan']
        vans = data['Van']
        buses = data['Bus']
        trucks = data['Truck']
        while done_vehicles < len(self.drivers):
            done_vehicles = 0
            if vehicle_counter < len(self.drivers):
                vehicle_type = vehicle_types[random.randint(0, len(vehicle_types) - 1)]
                driver_type = None
                driver = self.drivers[vehicle_counter]
                if isinstance(driver, RookieDriver):
                    driver_type = 'Rookie'
                elif isinstance(driver, ProfessionalDriver):
                    driver_type = 'Professional'
                elif isinstance(driver, HastyDriver):
                    driver_type = 'Hasty'
                    
                vehicle = None
                if vehicle_type == 'Sedan' and sedans > 0:
                    vehicle = Car(f'{vehicle_type}({driver_type}){vehicle_counter + 1}', driver, 15)
                    sedans = sedans - 1
                elif vehicle_type == 'Truck' and trucks > 0:
                    vehicle = Truck(f'{vehicle_type}({driver_type}){vehicle_counter + 1}', driver, 10)
                    trucks = trucks - 1
                elif vehicle_type == 'Van' and vans > 0:
                    vehicle = Van(f'{vehicle_type}({driver_type}){vehicle_counter + 1}', driver, 10)
                    vans = vans - 1
                elif vehicle_type == 'Bus' and buses > 0:
                    vehicle = Bus(f'{vehicle_type}({driver_type}){vehicle_counter + 1}', driver, 10)
                    buses = buses - 1
                else:
                    continue
                    
                self.vehicles.append(vehicle)
                vehicle_counter = vehicle_counter + 1
                self.road.lanes[random.randint(0, data['Lane'] -1)].addToLane(vehicle)
                vehicle.set_location([vehicle.lane.x_begin, vehicle.lane.y_begin, calculate_angle(vehicle.lane.x_begin, vehicle.lane, 'radian')])
                vehicle.set_cycle_offset(self.cycle_count)
                                      
            for lane in self.road.lanes:
                for vehicle in lane.vehicles:
                    vehicle.decide()
                    
                    # vehicle remove condition
                    if (vehicle.location[0] > roadXLength and vehicle in lane.vehicles) or vehicle.is_collided:
                        done_vehicles += 1
                        
            self.cycle_count += 1
    

In [23]:
def constrained_sum_sample_pos(n, total):
    """Return a randomly chosen list of n positive integers summing to total.
    Each such list is equally likely to occur."""

    dividers = sorted(random.sample(range(18, 61), n - 1))
    return [a - b for a, b in zip(dividers + [total], [0] + dividers)]
print(constrained_sum_sample_pos(5, 150))

[24, 2, 5, 28, 91]


In [24]:
from numpy.random import multinomial
print(multinomial(40, [1/4.] * 4))
#help(multinomial)

[13  5  9 13]


In [25]:
if __name__ == '__main__':
    
    GUI()
    print(data)
    #x1 = [0.0, 169.31, 331.17, 407.24, 470.98, 540.53, 604.66, 708.35, 791.48, 865.87, 934.41, 973.99, 1005.96, 1080.98, 1149.42, 1250.8, 1335.55, 1427.09, 1523.01, 1566.96, 1621.39, 1728.8, 1820.48, 1905.74, 1990.97, 2014.51, 2042.6, 2100.34, 2150.74]
    #y1 = [212.01, 189.54, 202.85, 242.31, 282.73, 325.98, 358.68, 348.78, 305.67, 261.61, 223.5, 188.17, 154.09, 60.25, 16.02, 2.0, 26.47, 57.54, 88.13, 101.67, 114.74, 97.81, 71.35, 73.11, 103.74, 124.06, 149.28, 189.89, 214.85]
    x1 = [0.0, 294.362, 575.772, 708.027, 818.846, 939.765, 1051.262, 1231.537, 1376.067, 1505.402, 1624.565, 1693.379, 1748.962, 1879.392, 1998.382, 2174.641, 2321.987, 2481.139, 2647.905, 2724.317, 2818.949, 3005.692, 3165.087, 3313.32, 3461.5, 3502.427, 3551.264, 3651.651, 3739.277]
    y1 = [368.601, 329.534, 352.675, 421.28, 491.554, 566.749, 623.601, 606.389, 531.438, 454.835, 388.577, 327.152, 267.901, 104.751, 27.852, 3.477, 46.021, 100.039, 153.223, 176.763, 199.487, 170.052, 124.049, 127.109, 180.362, 215.691, 259.538, 330.143, 373.538]
    x2 = [0.0, 73.03, 149.052, 220.401, 333.512, 429.922, 468.383, 572.744, 648.991, 714.07, 779.724, 908.12, 1062.561, 1118.112, 1248.619, 1449.083, 1675.221, 1894.536, 1948.879, 2107.028, 2301.447, 2475.354, 2551.683, 2642.273, 2695.714, 2738.704, 2806.263, 2923.226, 3168.318, 3311.387, 3461.258]
    y2 = [867.753, 840.705, 782.674, 680.506, 525.717, 390.251, 341.708, 246.322, 176.366, 104.934, 70.018, 61.391, 10.553, 4.098, 29.466, 24.22, 150.629, 177.329, 200.32, 343.88, 472.891, 631.901, 720.566, 844.475, 915.169, 961.52, 985.494, 970.023, 940.578, 917.444, 861.298]
    x3 = [0.0, 148.62, 261.462, 368.367, 535.468, 678.825, 912.668, 1081.522, 1212.539, 1539.784, 1597.061, 1702.111, 2008.815, 2128.205, 2288.656, 2564.845, 2940.372, 3162.914, 3243.751, 3407.058, 3729.693, 3795.986, 3887.92]
    y3 = [105.702, 130.404, 135.749, 122.102, 88.691, 56.666, 4.304, 15.584, 45.753, 5.609, 4.08, 40.327, 164.265, 221.196, 313.089, 358.23, 558.069, 575.346, 563.107, 501.362, 313.619, 247.163, 105.213]
    roads = [(x1, y1), (x2, y2), (x3, y3)]
    curve_approx_degree = 9
    lane_distance = 2
    road_name = data['Road Part']
    road_num = int(road_name[-1]) - 1
    road_tuple = roads[road_num]
    x = road_tuple[0]
    y = road_tuple[1]
    y = [-y_val for y_val in y]
    roadXLength = x[-1]
    
    base_lane_equation = np.poly1d(np.polyfit(x, y, curve_approx_degree))
    myderivation_model = np.polyder(base_lane_equation)
    x_der = [myderivation_model(x_val) for x_val in x]  # 1st-degree derivatives of pivot points on the base curve
    x_perp = [x + math.pi/2 for x in x_der]             # angles perpendicular to the derivative values
    
    lane_count = data['Lane']
    lane_equations = list()  # [0] is the right-most (bottom) lane
    x_begin_values = [x[0]] # [0] is the right-most (bottom) lane
    y_begin_values = [y[0]] # [0] is the right-most (bottom) lane
    for i in range(lane_count):
        if i == 0:
            lane_equations.append(base_lane_equation)
        else:
            x_adj = list()
            y_adj = list()
            for j, x_val in enumerate(x):
                x_delt = i*lane_distance*math.cos(x_perp[j])
                y_delt = i*lane_distance*math.sin(x_perp[j])
                x_adj.append(x_val+x_delt)
                y_adj.append(y[j]+y_delt)
                x_begin_values.append(x_val+x_delt)
                y_begin_values.append(y[j]+y_delt)
            lane_equations.append(np.poly1d(np.polyfit(x_adj, y_adj, curve_approx_degree)))
            
    lanes = list()
    for i in range(len(lane_equations)):
        lanes.append(Lane(lane_equations[i], x_begin_values[i], y_begin_values[i]))
    
    # Setting up lane relations
    # Dynamic version
    if lane_count > 1:
        for i in range(lane_count - 1):
            lanes[i].set_left_lane(lanes[i+1])
            lanes[i+1].set_right_lane(lanes[i])
            
    # Setting up the road
    road = Road('Road', lanes, lane_distance)
    for lane in lanes:
        lane.set_road(road)
        
    driver_vehicle_count = data['Driver']
    ages = [data['Mean Age']]* driver_vehicle_count
    
    # Create potholes and add to lanes.
    pothole_count = data['Defect']
    pothole_s = data['Small Defect']
    pothole_m = data['Medium Defect'] + pothole_s # Thinking as index
    pothole_l = data['Large Defect'] + pothole_m
    potholes = []
    
    for i in range(pothole_count):
        pothole_size = None
        if pothole_s > i:
            pothole_size = 's'
        elif pothole_m > i:
            pothole_size = 'm'
        elif pothole_l > i:
            pothole_size = 'l'
            
        lane = lanes[random.randint(0, len(lanes) - 1)]
        pothole_x_loc = random.uniform(x[0], x[-1] / 3)
        pothole_y_loc = lane.equation(pothole_x_loc)
        pothole_id = f'Pothole{i+1}'
        pothole = Pothole(pothole_id, pothole_x_loc, pothole_y_loc, pothole_size)
        pothole.set_lane(lane)
        lane.set_pothole(pothole)
        potholes.append(pothole)
        
    # Setting up traffic lights.
    light_num = data['Traffic Light']
    # Lead traffic light
    lead_lights = []
    for i in range(light_num):
        lead_light_x_loc = random.uniform(x[0], x[-1] / 2)
        lead_light_y_loc = lanes[0].equation(lead_light_x_loc)
        lead_light = TrafficLight(lead_light_x_loc, lead_light_y_loc, 0, frequency=20)
        lead_lights.append(lead_light)
        lane.set_light(lead_light)
        lead_light.set_lane(lanes[0])
        deriv_x = myderivation_model(lead_light_x_loc)
        for j in range(lane_count):
            if j == 0:
                continue
            lane = lanes[j]
            perpendicular_x_angle = math.pi / 2 + deriv_x
            copy_light_x_diff = j * road.lane_distance * math.cos(perpendicular_x_angle)
            copy_light_y_diff = j * road.lane_distance * math.sin(perpendicular_x_angle)
            copy_light_x = lead_light_x_loc + copy_light_x_diff
            copy_light_y = lead_light_y_loc + copy_light_y_diff
            copy_light = TrafficLight(copy_light_x, copy_light_y, 0, frequency=20)
            lane.set_light(copy_light)
            copy_light.set_lane(lane)
    
    # Setting up Drivers
    drivers = []
    driver_types = ['Rookie', 'Professional', 'Hasty']
    for driver_type in driver_types:
        driver_count = data[driver_type]
        for i in range(driver_count):
            driver = None
            if driver_type == "Rookie":
                driver = RookieDriver(ages[i])
            elif driver_type == "Professional":
                driver = ProfessionalDriver(ages[i])
            elif driver_type == "Hasty":
                driver = HastyDriver(ages[i])
            drivers.append(driver)
            
    random.shuffle(drivers)
            
    # Simulation
    print('• SIMULATION HAS STARTED •')
    start_time = time.time()
    
    simulation_engine = SimulationEngine.get_instance()
    simulation_engine.set_day_time(data['Time of Day'])
    simulation_engine.set_road(road)
    simulation_engine.set_drivers(drivers)
    simulation_engine.move_vehicles()
    vehicles = simulation_engine.get_vehicles()

    # Logging
    logger = Logger(road, 25, data['Time of Day'], potholes, lead_lights, x[0], y[0], x[-1], y[-1])
    
    for vehicle in vehicles:
        logger.addvehicle(vehicle)

    logger.outputToFile()
    print('• SIMULATION HAS ENDED •')
    print('Elapsed run time:', time.time() - start_time)
    print(f'{simulation_engine.cycle_count}')
    road_length_pixel = arclength(lane_equations[0], x[0], x[-1])
    road_length_meter = road_length_pixel * 1.792
    print(f'road pixel length: {road_length_pixel}')
    print(f'road meter length: {road_length_meter}')
    
    avg_speed_all = 0
    for vehicle in vehicles:
        avg_speed = mean(vehicle.speed_log())
        avg_speed_all += avg_speed
        
    avg_speed_all /= len(vehicles)
    avg_speed_all_mph = avg_speed_all * (1.8 / 1.6)
    print(avg_speed_all)
    #######
    ExcelReporter.report(vehicles, avg_speed_all_mph)
    Blender.run()

40 driver
2 defect
{'Mean Age': 30, 'Driver': 40, 'Rookie': 4, 'Professional': 26, 'Hasty': 10, 'Sedan': 25, 'Van': 5, 'Bus': 3, 'Truck': 7, 'Defect': 2, 'Small Defect': 1, 'Medium Defect': 0, 'Large Defect': 1, 'Traffic Light': 0, 'Road Part': 'Kopuz - Ozyegin 1', 'Lane': 3, 'Time of Day': 'D'}
• SIMULATION HAS STARTED •
• SIMULATION HAS ENDED •
Elapsed run time: 5.584064722061157
324
road pixel length: 4136.857765533704
road meter length: 7413.249115836397
1 1 Mean Age
1 2 30
2 1 Driver
2 2 40
3 1 Rookie
3 2 4
4 1 Professional
4 2 26
5 1 Hasty
5 2 10
6 1 Sedan
6 2 25
7 1 Van
7 2 5
8 1 Bus
8 2 3
9 1 Truck
9 2 7
10 1 Defect
10 2 2
11 1 Small Defect
11 2 1
12 1 Medium Defect
12 2 0
13 1 Large Defect
13 2 1
14 1 Traffic Light
14 2 0
15 1 Road Part
15 2 Kopuz - Ozyegin 1
16 1 Lane
16 2 3
17 1 Time of Day
17 2 D
1 8 Hasty
2 8 Van
3 8 30
1 9 Professional
2 9 Sedan
3 9 30
1 10 Professional
2 10 Truck
3 10 30
1 11 Professional
2 11 Van
3 11 30
1 12 Professional
2 12 Sedan
3 12 30
1 13 Profess